In [2]:
from scipy.special import roots_jacobi, eval_jacobi
import numpy as np
import scipy
import math

### Define functions for D operator:

In [3]:
def int_points(x_range, npts, dx, M):
    
    k = 0
    p_order = npts - 1
    xinterior, w = roots_jacobi(p_order - 1,1,1) # returns interior GLL nodes from range -1 and 1
    GLL_points = np.pad(xinterior, (1, 1), 'constant', constant_values=(-1, 1))
    GLL_dist = np.array([np.abs(GLL_points[0] - value) for value in GLL_points])
    
    return_array = np.zeros([M, npts])
    
    for i in x_range[:-1]:
        array = np.array([i])
        for j in GLL_dist[1:]: 
            scaled_point = ((j / 2) * (dx)) + i # Note: 2 here is the distance from -1 to 1
            array = np.concatenate((array, np.array([scaled_point])))
            
        return_array[k, :] = array
        k = k + 1
    
    return(return_array)

In [4]:
def vandermonde(npts_values, return_type):
    # For the monomial basis: 
    if return_type == 'Monomial':
        return(np.vander(npts_values, increasing = True))
    elif return_type == 'Legendre':
        return(np.polynomial.legendre.legvander(npts_values, len(npts_values) - 1))

In [5]:
def Ld_vandermonde(npts_values): 
    
    LP = np.polynomial.legendre
    v_matrix = np.polynomial.legendre.legvander(npts_values, len(npts_values)-1)
    d_v_matrix = np.zeros_like(v_matrix)

    for i in range(len(npts_values)):
        # Coefficients for the i-th Legendre polynomial (e.g., [0, 0, 1] for P_2)
        coeffs = np.zeros(len(npts_values))
        coeffs[i] = 1

        # Compute the derivative of the i-th Legendre polynomial
        deriv_coeffs = LP.legder(coeffs, m=1) # m=1 for first derivative

        # Evaluate the derivative at the points x
        d_v_matrix[:, i] = LP.legval(npts_values, deriv_coeffs)
        
    return(d_v_matrix)

In [6]:
def return_D(V1, V2):
    return(np.matmul(V2, np.linalg.inv(V1)))

In [7]:
# Initialize the elements and the Guassian function:

L = 12 # Denotes total length in [0, L]
M = 4 # Denotes number of elements 
x_range = np.arange(0, L + (L/M), (L/M)) # Define the element endpoints

npts = 4
p_order = npts - 1
xinterior, w = roots_jacobi(p_order - 1,1,1) # returns interior GLL nodes from range -1 and 1
GLL_points = np.pad(xinterior, (1, 1), 'constant', constant_values=(-1, 1))

# Note: The D matrix is only created once as it operates on the reference element
V1 = vandermonde(GLL_points, 'Legendre')
V2 = Ld_vandermonde(GLL_points)
D = return_D(V1, V2)

# The following is used in creating the init condition: 
npts = int_points(x_range, 4, L/M, M)

In [8]:
def quad_weights(npts_values):
    
    # Values in f_array result from exact integration:
    f_array = np.zeros(len(npts_values))
    f_array[0] = 2
    
    # Define basis matrix for Legendre polynomials: 
    V = np.polynomial.legendre.legvander(npts_values, len(npts_values) - 1).transpose()
    
    return(np.matmul(np.linalg.inv(V), f_array.transpose()))

## DG Global D Matrix:

In [11]:
C = (L / (2 * M))
-D[0, 0] + (1 / (2 * quad_weights(GLL_points)[-1] * C)) - (1 / (quad_weights(GLL_points)[-1] * C))

1.0

In [29]:
# Note: Have to come back to confirm the +/- signs; check indexing

def global_DG(D, M, p, L, quad_weights):
    
    array = np.zeros((M * (p + 1), M * (p + 1)))
    C = (L / (2 * M))
    row_i = 0
    col_i = 0
    
    for i in np.arange(M):
        if i == 0:
            array[row_i : row_i + (p+1), col_i : col_i + (p+1)] = (-D * (1 / C))
            
            # Right boundary correction:
            array[row_i + p, row_i + p] = (-D[-1, -1] * (1 / C)) - (1 / (2 * quad_weights[-1] * C)) + (1 / (quad_weights[-1] * C))
            array[row_i + p, row_i + p + 1] = -(1 / (2 * quad_weights[-1] * C))
            
            # Left boundary correction:
            array[row_i, row_i] = (-D[0, 0] * (1 / C)) + (1 / (2 * quad_weights[0] * C)) - (1 / (quad_weights[0] * C))
            
            # Periodic boundaries:
            array[0, -1] = (1 / (2 * quad_weights[0] * C))
            
        elif i == (M-1):
            array[row_i : row_i + (p+1), col_i : col_i + (p+1)] = (-D * (1 / C))
            
            # Left boundary correction:
            array[row_i, row_i] = (-D[0, 0] * (1 / C)) + (1 / (2 * quad_weights[0] * C)) - (1 / (quad_weights[0] * C))
            array[row_i, row_i - 1] = (1 / (2 * quad_weights[0] * C))
            
            # Right boundary correction:
            array[row_i + p, row_i + p] = (-D[-1, -1] * (1 / C)) - (1 / (2 * quad_weights[-1] * C)) + (1 / (quad_weights[-1] * C))
            
            # Periodic boundaries:
            array[-1, 0] = -(1 / (2 * quad_weights[-1] * C))
            
        else:
            array[row_i : row_i + (p+1), col_i : col_i + (p+1)] = (-D * (1 / C))
            
            # Right boundary correction:
            array[row_i + p, row_i + p] = (-D[-1, -1] * (1 / C)) - (1 / (2 * quad_weights[-1] * C)) + (1 / (quad_weights[-1] * C))
            array[row_i + p, row_i + p + 1] = -(1 / (2 * quad_weights[-1] * C))
        
            # Left boundary correction:
            array[row_i, row_i] = (-D[0, 0] * (1 / C)) + (1 / (2 * quad_weights[0] * C)) - (1 / (quad_weights[0] * C))
            array[row_i, row_i - 1] = (1 / (2 * quad_weights[0] * C))
        
        row_i = row_i + (p+1)
        col_i = col_i + (p+1)
        
    return(array)


In [32]:
np.linalg.eigvals(global_DG(D, M, p_order, L, quad_weights(GLL_points)))

array([-6.66133815e-16+2.99799668j, -6.66133815e-16-2.99799668j,
       -5.55111512e-16+2.87346072j, -5.55111512e-16-2.87346072j,
       -2.22044605e-16+2.5819889j , -2.22044605e-16-2.5819889j ,
       -7.21644966e-16+2.279092j  , -7.21644966e-16-2.279092j  ,
       -1.66533454e-16+1.2424426j , -1.66533454e-16-1.2424426j ,
       -1.11022302e-16+1.03757255j, -1.11022302e-16-1.03757255j,
        1.80411242e-16+0.52353792j,  1.80411242e-16-0.52353792j,
       -3.28003888e-16+0.j        ,  9.99829959e-18+0.j        ])

### Application to 1D Advection:

In [14]:
def init_Guassian(input_array, mu, sigma): 
    
    return_array = np.zeros_like(input_array)
    
    for i in np.arange(input_array.shape[0]):
        
        numerator = np.exp(-(input_array[i] - mu)**2 / (2 * sigma**2))
        denominator = sigma * np.sqrt(2 * np.pi)
        return_array[i, :] = numerator / denominator 
        
    return(return_array)

In [18]:
u = init_Guassian(npts, (L/2), 1)
u_G = u.reshape(-1)
u_G

array([6.07588285e-09, 6.23678826e-07, 2.61194096e-04, 4.43184841e-03,
       4.43184841e-03, 3.78104741e-02, 2.82886947e-01, 3.98942280e-01,
       3.98942280e-01, 2.82886947e-01, 3.78104741e-02, 4.43184841e-03,
       4.43184841e-03, 2.61194096e-04, 6.23678826e-07, 6.07588285e-09])

In [30]:
t_steps = 12
dt = 0.10

u_G = init_Guassian(npts, (L/2), 1).reshape(-1)

for i in np.arange(0, t_steps, dt):
    
    u_mid = np.zeros_like(u)
    u_n = np.zeros_like(u)
    
    # Half time step:
    u_mid = u_G + ((dt / 2) * np.matmul(global_DG(D, M, p_order, L, quad_weights(GLL_points)), u_G))
    # Full time step:
    u_n = u_G + ((dt) * np.matmul(global_DG(D, M, p_order, L, quad_weights(GLL_points)), u_mid))
    
    # Prepare for the next timestep:
    u_G = u_n

In [31]:
u_G # It works! - compare to element-by-element calculation 

array([ 0.088567  , -0.02167252, -0.0776931 ,  0.04841133,  0.04698194,
        0.12319314,  0.24800313,  0.30239349,  0.29467319,  0.29250982,
        0.09871945, -0.02156745, -0.0234518 , -0.07653663,  0.05353969,
        0.08888182])

## CG Global D Matrix:

In [33]:
def global_CG(D, M, p):
    
    row_i = 0
    col_i = 0
    edg_i = p
    C = (L / (2 * M))
    array = np.zeros(((M * p), (M * p)))
    
    for i in np.arange(M):
        if i == 0:
            array[row_i : row_i + (p+1), 
                  col_i : col_i + (p+1)] = (-D * (1 / C))
            
            # Periodic Boundary:
            array[0, -3:] = (-D[-1, :3] * (1 / C))
            
            # Boundary Correction:
            array[0, 0] = array[0, 0] - (D[-1, -1] * (1 / C))
            array[0, :] = (array[0, :] / 2) # For centered average of u', divide all terms by 2
            
        elif i == M-1:
            array[row_i : row_i + (p+1), 
                  col_i : col_i + (p+1)] = (-D[:3, :3] * (1 / C))
            
            # Periodic Boundary:
            array[-3:, 0] = (-D[:3 , -1] * (1 / C))
            
            # Boundary Correction:
            array[edg_i, edg_i] = (-D[0, 0] - D[-1, -1]) * (1 / C)
            array[edg_i, :] = (array[edg_i, :] / 2) # For centered average of u', divide all terms by 2
    
        else:
            array[row_i : row_i + (p+1), 
                  col_i : col_i + (p+1)] = (-D * (1 / C))
            
            # Boundary Correction:
            array[edg_i, edg_i] = (-D[0, 0] - D[-1, -1]) * (1 / C)
            array[edg_i, :] = (array[edg_i, :] / 2) # For centered average of u', divide all terms by 2
            edg_i = edg_i + p
            
        row_i = row_i + (p)
        col_i = col_i + (p)    
        
    return(array)
        

In [44]:
np.linalg.eigvals(global_CG(D, M, p_order))

array([ 5.88797394e-17+1.82574186j,  5.88797394e-17-1.82574186j,
        1.66533454e-16+1.55510372j,  1.66533454e-16-1.55510372j,
       -4.16333634e-17+1.364998j  , -4.16333634e-17-1.364998j  ,
        6.68713706e-17+1.05409255j,  6.68713706e-17-1.05409255j,
        2.77555756e-17+0.52343904j,  2.77555756e-17-0.52343904j,
       -1.06403591e-16+0.j        ,  1.84777590e-17+0.j        ])

### Application to 1D Advection:

In [46]:
t_steps = 12
dt = 0.10

u_G = np.array([arr[:3] for arr in init_Guassian(npts, (L/2), 1)]).reshape(-1)

for i in np.arange(0, t_steps, dt):
    
    u_mid = np.zeros_like(u)
    u_n = np.zeros_like(u)
    
    # Half time step:
    u_mid = u_G + ((dt / 2) * np.matmul(global_CG(D, M, p_order), u_G))
    # Full time step:
    u_n = u_G + ((dt) * np.matmul(global_CG(D, M, p_order), u_mid))
    
    # Prepare for the next timestep:
    u_G = u_n

In [48]:
u_G

array([-0.04796433,  0.02177871,  0.01388166, -0.0428971 ,  0.04040688,
        0.28244533,  0.37306751,  0.29033937,  0.05559084, -0.02030702,
       -0.00376252, -0.00039902])

## Eigenvector Analysis:

In [58]:
np.linalg.eig(global_DG(D, M, p_order, L, quad_weights(GLL_points)))[1][0].imag

array([-3.39840453e-01,  3.39840453e-01, -3.41093288e-01,  3.41093288e-01,
        3.05311332e-16, -3.05311332e-16,  0.00000000e+00, -0.00000000e+00,
        1.42247325e-16, -1.42247325e-16, -2.07758194e-01,  2.07758194e-01,
       -2.25460525e-01,  2.25460525e-01,  0.00000000e+00,  0.00000000e+00])

In [68]:
test = np.fft.fft(np.linalg.eig(global_DG(D, M, p_order, L, quad_weights(GLL_points)))[1][1])
test

array([-0.65166115+0.j        , -0.50420412-0.34419533j,
       -0.16424208+0.52503564j, -0.62561325-0.33246373j,
        0.59942969-0.77505386j,  0.40303633+0.12736262j,
       -0.59583899+0.57800859j, -0.01423908-0.17361035j,
        0.17562417-0.38594239j,  0.46802374-0.18959984j,
        0.49323212+1.07717976j, -0.39943321-0.42904402j,
        0.24628478+0.42190895j, -0.82719031-0.20372927j,
        0.52135938-0.07392762j, -0.57645112+0.44012337j])

In [67]:
((test.imag) ** 2) + ((test.real) ** 2)

array([0.42466225, 0.37269222, 0.30263789, 0.50192407, 0.96002445,
       0.17865952, 0.68911804, 0.0303433 , 0.17979538, 0.25499432,
       1.40359415, 0.34362565, 0.23866336, 0.72574943, 0.27728089,
       0.52600448])